# Lab 3.1 - Algorytmy genetyczne

In [1]:
import numpy as np
import random

In [21]:
class evolution:
    
    def __init__(self, population_size, chr_size, target_convert_function, init_method = 'binary'):
        self.n = population_size
        self.m = chr_size
        self.target = target_convert_function
        self.best = None
        self.generation = []
        
        # initialize random generation
        if init_method == 'binary':
            for i in range(self.n):
                self.generation.append(np.random.choice([0, 1], size=(self.m), p=[0.5,0.5]))
        else:
            for i in range(self.n):
                self.generation.append(np.random.uniform(-8, 8, self.m))
        self.generation = np.asarray(self.generation)

    def cross(self,x,y):
        """
        krzyżowanie jednopunktowe, punkt podziału jest losowy z orzedziału [1,len-1]
        """
        
        split = random.randint(1,len(x)-1)
        x_1 = x[0:split]
        x_2 = x[split:len(x)]
        y_1 = y[0:split]
        y_2 = y[split:len(y)]
        child_1 = [*x_1, *y_2]
        child_2 = [*y_1, *x_2]
        return [child_1, child_2]
    
    def mutate(self, generation, p=0.2):
        """
        mutacja gaussowska, każdy gen jest modyfikowany o zmienną standardową normalną z prawd. p
        """
        for i, osobnik in enumerate(generation):
            for j, gen in enumerate(osobnik):
                decision = random.uniform(0,1)
                if decision < p:
                    generation[i][j] += np.random.normal(0,1)
        return generation
    
    def evolve(self):
        """
        główna funkcja przeprowadzająca ewolucję i szukająca minimum
        """
        it = 0
        current_best_value = -1
        while True:
            it +=1
            
            # evaluate
            values = np.asarray([self.target(self.generation[i]) for i in range(self.n)])
            for i, value in enumerate(values):
                if value > current_best_value:
                    self.best = self.generation[i]
                    current_best_value = value
                        
            # check stop condition
            if (self.target(self.best) >= 590):
                #return self.target(self.best)
                #return it
                return (it, self.target(self.best))
            
            # select parents for new generation
            prob = values/values.sum()
            choices = np.random.choice([i for i in range(self.n)], size=self.n, p=prob)
            generation_parents = [self.generation[choice] for choice in choices]
            
            # cross
            next_generation = []
            p_cross = 0.7
            
            while(len(next_generation) < self.n):
                decision = random.uniform(0,1)
                if decision < 0.7:
                    # cross
                    parents = random.sample([i for i in range(self.n)], 2)
                    children = self.cross(generation_parents[parents[0]], generation_parents[parents[1]])
                    next_generation.append(children[0])
                    next_generation.append(children[1])
                else:
                    # random osobnik
                    next_generation.append([*generation_parents[random.sample([i for i in range(self.n)], 1)[0]]])
                    
            if len(next_generation) > self.n:
                next_generation = next_generation[0:self.n]
                    
            # mutate
            next_generation = self.mutate(next_generation)
            
            # update
            self.generation = np.asarray(next_generation)



## Funkcja kwadratowa

In [3]:
def target_1(x,y,z):
    return (x**2) + (y**2) + 2*(z**2)

In [14]:
def eval_1(osobnik, arguments = False):
    target = target_1(osobnik[0],osobnik[1],osobnik[2])
    if arguments:
        return (x,y,z,1024-target)
    return 1024 - target   

In [5]:
evo = evolution(100, 3, eval_1, init_method='dupa')

In [6]:
evo.evolve()

(165, 1023.2923797685013)

## Funkcja Rastrigina

In [7]:
def rastrigin_5(x, A=10):
    """funckja rastrigina"""
    return (A*len(x) + np.sum(x**2 - A*np.cos(2*np.pi*x)))

In [13]:
rastrigin_5(np.array([-10,-10,-10,-10,-10]))

500.0

In [15]:
def eval_2(osobnik, arguments = False):
    target = rastrigin_5(osobnik)
    if arguments:
        return (osobnik, 600-target)
    return 600 - target   

In [24]:
evo = evolution(100, 3, eval_2, init_method='dupa')
evo.evolve()

(18, 594.0536793700562)